# DD2424 Project in Deep Learning in Data Science

## Imports

In [ ]:
from functions import *
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow as tf
import os

## Vanilla RNN

### Load and Preprocess Data

In [ ]:
training_data_filename = '../Dataset/Training/edgar.txt'
book_data = np.array(load_data(training_data_filename))
book_chars = np.unique(book_data)

char_to_ind = {ch:i for i,ch in enumerate(book_chars)}
ind_to_char = {i:ch for i,ch in enumerate(book_chars)}
k = book_chars.shape[0]
m = 100
eta = 0.1
seq_length = 25
sig = 0.01


### Train

In [ ]:
rnn = RNN(m, k, eta, seq_length, sig)
h0 = np.zeros((m, 1))
max_iter = 200000
epochs = 10
smoothloss_list = []
loss_list = []
iteration = 0
smoothloss = 0
sentences = []
for i in range(epochs):
    rnn.hprev = np.zeros((m, 1))
    for e in range(0, book_data.shape[0]-seq_length-1, seq_length):
        X_chars = book_data[e:e+seq_length]
        Y_chars = book_data[e+1:e+seq_length+1]
        X = one_hot_encoding(X_chars, char_to_ind, k)
        Y = one_hot_encoding(Y_chars, char_to_ind, k)
        loss = rnn.adagrad(X, Y, h0, iteration)
        if smoothloss == 0:
            smoothloss = loss
        smoothloss = 0.999*smoothloss + 0.001*loss
     
        if iteration % 10000 == 1:
            print('Iteration: {}, Loss: {} '.format(iteration, smoothloss))
            y = rnn.synthetize(rnn.hprev, X[:, 0], 200)
            sentence = one_hot_decoding(y, ind_to_char)
            print(sentence + "\n")
            #sentences.append(sentence)
            smoothloss_list.append(smoothloss)
            loss_list.append(loss)
        
        iteration += 1
        if iteration>max_iter:
            break


### Evaluate and plot

In [ ]:

with open(training_data_filename,encoding='cp850',mode='r') as file:
    validation_text = file.read()
start_char = "T"
start_char_onehot = one_hot_encoding(start_char, char_to_ind, k)
generated_text_vanilla_onehot = rnn.synthetize(rnn.hprev, start_char_onehot, 1000)
generated_text_vanilla = start_char + one_hot_decoding(generated_text_vanilla_onehot, ind_to_char)
print(generated_text_vanilla)

# Calculate performance metrics for generated text
nmax = 4
fraction_correct_words, bleu_score = measure_bleu(text_generated=generated_text_vanilla, text_val=validation_text, n_max=nmax)
repetition_score = measure_diversity(text_generated=generated_text_vanilla, n_max=nmax)
print("\n loss function", loss_list)
print("\n fraction of correctly spelled words: {} \n Bleu score: {} \n Repetition score: {}".format(fraction_correct_words, bleu_score, repetition_score))



fig = px.line(smoothloss_list, title='Smoothed loss over epochs', width=600)
fig.update_layout(showlegend=False)
fig.update_yaxes(title_text="smoothed loss")
fig.update_xaxes(title_text="iteration step, in multiples of 10k")
fig.show()



## LSTM

### Load and Preprocess Data

In [ ]:

training_data_filename = '../Dataset/Training/edgar.txt'
book_data = np.array(load_data(training_data_filename,remove_footnotes=False))
with open(training_data_filename,encoding='cp850',mode='r') as f:
   words = f.read().split()
with open(training_data_filename,encoding='cp850',mode='r') as file:
    validation_text = file.read()

book_chars = np.unique(book_data)
char_to_ind = {ch:i for i,ch in enumerate(book_chars)}
ind_to_char = {i:ch for i,ch in enumerate(book_chars)}
k = book_chars.shape[0]

# Hyperparameters
batch_size = 25 # [25, 75, 125]
seq_length = 25 # [25, 75, 125]
learningrate = 0.1 # [0.1, 0.01, 0.001]
book_data_ind = np.array([char_to_ind[c] for c in book_data])

# Split data into sequences
char_dataset = tf.data.Dataset.from_tensor_slices(book_data_ind)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(data):
    input = data[:-1]
    target = data[1:]
    return input, target

# Split data into X, Y
dataset = sequences.map(split_input_target)

# This organizes the data into groups of sequences. batch_size denotes the number of sequences in a batch, and seq_length denotes the number of characters in a sequence.
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)


### Build and train model

In [ ]:

modelpathname = 'v6_1'
#def loss(Y_true,Y_pred):
#    return tf.keras.losses.SparseCategoricalCrossentropy(from)


# Length of the vocabulary in chars.
vocab_size = len(book_chars)

# The embedding dimension.
embedding_dim = 256 

# Number of RNN units.
u = [50, 100, 200, 500, 1000]
rnn_units = 200
n_epochs = 5


def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    m = tf.keras.Sequential()
    m.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, batch_input_shape=[batch_size, None]))
    m.add(tf.keras.layers.BatchNormalization(synchronized=True))
    m.add(tf.keras.layers.LSTM(units=rnn_units, return_sequences=True, stateful=True, recurrent_initializer=tf.keras.initializers.GlorotNormal()))
    m.add(tf.keras.layers.LSTM(units=rnn_units, return_sequences=True, stateful=True, recurrent_initializer=tf.keras.initializers.GlorotNormal()))
    m.add(tf.keras.layers.Dense(vocab_size))
    #m.summary()
    return m

m = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

# try the model
for i_ex, t_ex in dataset.take(1):
    example_pred = m(i_ex)  # this step builds the model
    sampled_indices = tf.random.categorical(logits=example_pred[0], num_samples=1)
    sample_ind_1d = tf.squeeze(input=sampled_indices, axis=-1).numpy()
print('Input:\n', repr(''.join([ind_to_char[c] for c in i_ex.numpy()[0]])))
print('Next char prediction:\n', repr(''.join([ind_to_char[c] for c in sample_ind_1d])))  

# Directory where the checkpoints will be saved.
current_dir_path = os.getcwd()
checkpoint_dir = current_dir_path + "\\tmp\\" + modelpathname+ "\\training_checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch:02d}.hdf5')
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True, 
    save_best_onl =True
)


# Specify update rule and compile model
adam_opt = tf.keras.optimizers.Adam(learning_rate=learningrate)
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
m.compile(optimizer=adam_opt, loss=loss_func)

# train
history = m.fit(x=dataset, epochs=n_epochs, callbacks=[checkpoint_callback])



### Generate text

In [ ]:
# Find latest checkpoint file, because tf.train.latest_checkpoint(dir) doesn't work for some reason
latest_epoch = 0
latest_checkpoint_file = ""
for file in os.listdir(checkpoint_dir):
    e = int(file.split("_")[1].split(".")[0])
    if e>latest_epoch:
        latest_epoch = e
        latest_checkpoint_file = file
print(latest_checkpoint_file)
#latest_checkpoint_file = "ckpt_01.hdf5"

simplified_batch_size = 1
m = build_model(vocab_size, embedding_dim, rnn_units, simplified_batch_size)
m.load_weights(checkpoint_dir + "/" + latest_checkpoint_file)
m.build(tf.TensorShape([simplified_batch_size, None]))

gen_text = generate_text(m, "The ", 1000, char_to_ind, ind_to_char)


### Evaluate and plot

In [ ]:
print(history.history['loss'])
d = get_n_grams(gen_text, 1)
fig = px.line(history.history['loss'], title='Loss over epochs', width=600)
#fig = px.scatter(history.history['loss'], title='Loss over epochs')
fig.show()

correctwords = 0
for word in d: 
    if word in words: 
        #print(word)
        correctwords += 1
print(gen_text)


print("\n Correct % words: {}".format(correctwords/len(d)))

# Calculate performance metrics for generated text
fraction_correct_words, bleu_score2 = measure_bleu(text_generated=gen_text, text_val=validation_text, n_max=2)
fraction_correct_words, bleu_score4 = measure_bleu(text_generated=gen_text, text_val=validation_text, n_max=4)
repetition_score2 = measure_diversity(text_generated=gen_text, n_max=2)
repetition_score4 = measure_diversity(text_generated=gen_text, n_max=4)
print("fraction of correctly spelled words: {} \n Bleu score2: {} \n Repetition score2: {}".format(fraction_correct_words, bleu_score2, repetition_score2))
print("\n Bleu score4: {} \n Repetition score4: {}".format(bleu_score4, repetition_score4))